# San Diego Homelessness Service Gap Analysis

This notebook explores the service gap scoring algorithm used in the San Diego Homelessness Dashboard.
We'll analyze different weighting schemes and visualize how they affect the final gap scores.

## Service Gap Score Components

The current algorithm calculates a score (0-100) based on:
1. **Poverty Rate** (0-50 points): Higher poverty increases gap score
2. **Population Density** (0-25 points): More people may need more services
3. **Service Availability** (0-15 points): Fewer services increase gap score
4. **Geographic Access** (0-10 points): Distance to nearest service

Higher scores indicate greater need for additional homeless services.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
from shapely.geometry import Point, Polygon
from scipy.spatial.distance import cdist
import warnings
warnings.filterwarnings('ignore')

# Set up plotting style
plt.style.use('default')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 8)

print("Libraries loaded successfully!")

## Load and Process Data

In [ ]:
# Load the datasets
print("Loading data...")

# Load demographic data
acs_data = pd.read_csv('public/data/acs2023_hackathon.csv')
print(f"ACS Data: {len(acs_data)} zip codes")

# Load homeless services
with open('public/data/homeless_services_hackathon.json', 'r') as f:
    services_data = json.load(f)
services_df = pd.DataFrame(services_data)
print(f"Homeless Services: {len(services_df)} locations")

# Load zip code boundaries
zip_gdf = gpd.read_file('public/data/sd_zipcodes.geojson')
print(f"Zip Code Boundaries: {len(zip_gdf)} polygons")

# Process demographic data
acs_processed = acs_data.copy()
acs_processed['zipCode'] = acs_processed['Zip Code'].astype(str)
acs_processed['population'] = acs_processed['Population'].fillna(0)
acs_processed['povertyPopulation'] = acs_processed['Population in Poverty'].fillna(0)
acs_processed['povertyRate'] = (
    acs_processed['povertyPopulation'] / acs_processed['population'] * 100
).fillna(0)
acs_processed['medianIncome'] = acs_processed['Median Income'].fillna(0)

# Process services data
services_clean = services_df[
    (services_df['latitude'].notna()) & 
    (services_df['longitude'].notna())
].copy()
services_clean['lat'] = pd.to_numeric(services_clean['latitude'], errors='coerce')
services_clean['lng'] = pd.to_numeric(services_clean['longitude'], errors='coerce')
services_clean = services_clean.dropna(subset=['lat', 'lng'])

print(f"\nProcessed:")
print(f"- Demographics: {len(acs_processed)} zip codes")
print(f"- Services: {len(services_clean)} valid locations")
print(f"- Poverty rate range: {acs_processed['povertyRate'].min():.1f}% - {acs_processed['povertyRate'].max():.1f}%")

## Current Service Gap Algorithm

Let's implement the current algorithm from the JavaScript code:

In [ ]:
def calculate_gap_score_current(poverty_rate, population, service_count, nearest_distance):
    """
    Current algorithm from the JavaScript implementation
    """
    score = 0
    
    # Poverty component (0-50 points)
    score += min(poverty_rate * 2, 50)
    
    # Population component (0-25 points)
    population_score = min((population / 1000) * 5, 25)
    score += population_score
    
    # Service availability component (0-15 points)
    service_score = max(15 - (service_count * 3), 0)
    score += service_score
    
    # Distance component (0-10 points)
    if nearest_distance is not None:
        distance_score = min(nearest_distance * 2, 10)
        score += distance_score
    else:
        score += 10
    
    return min(score, 100)

# Test the function
test_score = calculate_gap_score_current(25.0, 50000, 2, 3.5)
print(f"Test gap score: {test_score:.1f}")

# Break down the components
print("\nScore breakdown for test case:")
print(f"- Poverty (25% rate): {min(25 * 2, 50):.1f} points")
print(f"- Population (50k): {min((50000 / 1000) * 5, 25):.1f} points")
print(f"- Services (2 count): {max(15 - (2 * 3), 0):.1f} points")
print(f"- Distance (3.5 miles): {min(3.5 * 2, 10):.1f} points")
print(f"- Total: {test_score:.1f}/100")

## Calculate Service Counts and Distances

For each zip code, we need to count services and find the nearest service:

In [ ]:
# Merge demographic data with zip code boundaries
zip_gdf['zipCode'] = zip_gdf['zip'].astype(str)
merged_gdf = zip_gdf.merge(acs_processed, on='zipCode', how='left')

print(f"Merged GeoDataFrame: {len(merged_gdf)} zip codes")
print(f"With demographic data: {merged_gdf['population'].notna().sum()} zip codes")

# Create GeoDataFrame for services
services_geometry = [Point(lng, lat) for lng, lat in zip(services_clean['lng'], services_clean['lat'])]
services_gdf = gpd.GeoDataFrame(services_clean, geometry=services_geometry, crs='EPSG:4326')

# Ensure same CRS
merged_gdf = merged_gdf.to_crs('EPSG:4326')

print(f"Services GeoDataFrame: {len(services_gdf)} locations")

In [ ]:
def calculate_service_metrics(zip_gdf, services_gdf):
    """
    Calculate service count and nearest distance for each zip code
    """
    results = []
    
    for idx, zip_row in zip_gdf.iterrows():
        zip_code = zip_row['zipCode']
        zip_geom = zip_row.geometry
        
        if pd.isna(zip_row['population']) or zip_row['population'] == 0:
            results.append({
                'zipCode': zip_code,
                'serviceCount': 0,
                'nearestDistance': None
            })
            continue
        
        # Count services within zip code
        services_in_zip = services_gdf[services_gdf.geometry.within(zip_geom)]
        service_count = len(services_in_zip)
        
        # Calculate distance to nearest service
        zip_centroid = zip_geom.centroid
        
        if len(services_gdf) > 0:
            # Convert to projected CRS for distance calculation (meters)
            temp_zip = gpd.GeoDataFrame([zip_centroid], columns=['geometry'], crs='EPSG:4326')
            temp_services = services_gdf[['geometry']].copy()
            
            # Project to California State Plane for accurate distance
            temp_zip_proj = temp_zip.to_crs('EPSG:3310')  # California Albers
            temp_services_proj = temp_services.to_crs('EPSG:3310')
            
            distances = temp_services_proj.geometry.distance(temp_zip_proj.geometry.iloc[0])
            nearest_distance_meters = distances.min()
            nearest_distance_miles = nearest_distance_meters / 1609.34  # Convert to miles
        else:
            nearest_distance_miles = None
        
        results.append({
            'zipCode': zip_code,
            'serviceCount': service_count,
            'nearestDistance': nearest_distance_miles
        })
    
    return pd.DataFrame(results)

print("Calculating service metrics for each zip code...")
service_metrics = calculate_service_metrics(merged_gdf, services_gdf)

print(f"\nService Metrics Summary:")
print(f"- Average services per zip: {service_metrics['serviceCount'].mean():.1f}")
print(f"- Max services in one zip: {service_metrics['serviceCount'].max()}")
print(f"- Zip codes with no services: {(service_metrics['serviceCount'] == 0).sum()}")
print(f"- Average distance to nearest service: {service_metrics['nearestDistance'].mean():.1f} miles")

service_metrics.head()

## Calculate Gap Scores and Analyze Results

In [ ]:
# Merge all data together
analysis_df = merged_gdf.merge(service_metrics, on='zipCode', how='left')

# Calculate gap scores
analysis_df['gapScore'] = analysis_df.apply(
    lambda row: calculate_gap_score_current(
        row['povertyRate'], 
        row['population'], 
        row['serviceCount'], 
        row['nearestDistance']
    ) if pd.notna(row['population']) and row['population'] > 0 else np.nan,
    axis=1
)

# Filter to valid data
valid_data = analysis_df.dropna(subset=['gapScore', 'population', 'povertyRate'])

print(f"Gap Score Analysis:")
print(f"- Valid zip codes: {len(valid_data)}")
print(f"- Gap score range: {valid_data['gapScore'].min():.1f} - {valid_data['gapScore'].max():.1f}")
print(f"- Average gap score: {valid_data['gapScore'].mean():.1f}")
print(f"- High gap areas (>80): {(valid_data['gapScore'] > 80).sum()}")

# Show top 10 highest gap scores
print("\nTop 10 Highest Gap Scores:")
top_gaps = valid_data.nlargest(10, 'gapScore')[[
    'zipCode', 'community', 'gapScore', 'povertyRate', 'population', 
    'serviceCount', 'nearestDistance'
]]
print(top_gaps.to_string(index=False))

## Visualize Current Gap Scores

In [ ]:
# Create visualization dashboard
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('San Diego Homelessness Service Gap Analysis', fontsize=16, fontweight='bold')

# 1. Gap Score Distribution
axes[0,0].hist(valid_data['gapScore'], bins=20, alpha=0.7, color='skyblue', edgecolor='black')
axes[0,0].axvline(valid_data['gapScore'].mean(), color='red', linestyle='--', 
                  label=f'Mean: {valid_data["gapScore"].mean():.1f}')
axes[0,0].set_xlabel('Gap Score')
axes[0,0].set_ylabel('Number of Zip Codes')
axes[0,0].set_title('Distribution of Gap Scores')
axes[0,0].legend()
axes[0,0].grid(True, alpha=0.3)

# 2. Gap Score vs Poverty Rate
scatter = axes[0,1].scatter(valid_data['povertyRate'], valid_data['gapScore'], 
                           c=valid_data['serviceCount'], cmap='viridis', alpha=0.7)
axes[0,1].set_xlabel('Poverty Rate (%)')
axes[0,1].set_ylabel('Gap Score')
axes[0,1].set_title('Gap Score vs Poverty Rate\n(Color = Service Count)')
plt.colorbar(scatter, ax=axes[0,1], label='Service Count')
axes[0,1].grid(True, alpha=0.3)

# 3. Service Count Distribution
service_counts = valid_data['serviceCount'].value_counts().sort_index()
axes[1,0].bar(service_counts.index, service_counts.values, alpha=0.7, color='lightcoral')
axes[1,0].set_xlabel('Number of Services')
axes[1,0].set_ylabel('Number of Zip Codes')
axes[1,0].set_title('Distribution of Service Counts')
axes[1,0].grid(True, alpha=0.3)

# 4. Distance vs Gap Score
distance_data = valid_data.dropna(subset=['nearestDistance'])
axes[1,1].scatter(distance_data['nearestDistance'], distance_data['gapScore'], 
                  alpha=0.6, color='orange')
axes[1,1].set_xlabel('Distance to Nearest Service (miles)')
axes[1,1].set_ylabel('Gap Score')
axes[1,1].set_title('Gap Score vs Distance to Nearest Service')
axes[1,1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Component Analysis

Let's break down how each component contributes to the final score:

In [ ]:
def calculate_score_components(df):
    """
    Calculate individual components of the gap score
    """
    components = pd.DataFrame()
    components['zipCode'] = df['zipCode']
    
    # Poverty component (0-50)
    components['poverty_component'] = np.minimum(df['povertyRate'] * 2, 50)
    
    # Population component (0-25)
    components['population_component'] = np.minimum((df['population'] / 1000) * 5, 25)
    
    # Service component (0-15)
    components['service_component'] = np.maximum(15 - (df['serviceCount'] * 3), 0)
    
    # Distance component (0-10)
    components['distance_component'] = np.where(
        df['nearestDistance'].notna(),
        np.minimum(df['nearestDistance'] * 2, 10),
        10
    )
    
    components['total_score'] = (
        components['poverty_component'] + 
        components['population_component'] + 
        components['service_component'] + 
        components['distance_component']
    )
    
    return components

components_df = calculate_score_components(valid_data)

# Component statistics
print("Component Analysis:")
print("\nAverage contribution by component:")
for col in ['poverty_component', 'population_component', 'service_component', 'distance_component']:
    avg_contribution = components_df[col].mean()
    pct_of_total = (avg_contribution / components_df['total_score'].mean()) * 100
    print(f"- {col.replace('_', ' ').title()}: {avg_contribution:.1f} points ({pct_of_total:.1f}% of total)")

# Correlation analysis
print("\nComponent Correlations:")
correlation_matrix = components_df[[
    'poverty_component', 'population_component', 
    'service_component', 'distance_component', 'total_score'
]].corr()
print(correlation_matrix.round(3))

In [ ]:
# Visualize component contributions
fig, axes = plt.subplots(2, 2, figsize=(16, 10))
fig.suptitle('Service Gap Score Component Analysis', fontsize=16, fontweight='bold')

component_cols = ['poverty_component', 'population_component', 'service_component', 'distance_component']
component_names = ['Poverty', 'Population', 'Service Availability', 'Geographic Access']
colors = ['red', 'blue', 'green', 'orange']

for i, (col, name, color) in enumerate(zip(component_cols, component_names, colors)):
    ax = axes[i//2, i%2]
    ax.hist(components_df[col], bins=15, alpha=0.7, color=color, edgecolor='black')
    ax.axvline(components_df[col].mean(), color='darkred', linestyle='--', 
               label=f'Mean: {components_df[col].mean():.1f}')
    ax.set_xlabel('Score Points')
    ax.set_ylabel('Number of Zip Codes')
    ax.set_title(f'{name} Component Distribution')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Stacked bar chart of components for top 15 zip codes
top_15 = components_df.nlargest(15, 'total_score')
top_15_with_names = top_15.merge(valid_data[['zipCode', 'community']], on='zipCode')

fig, ax = plt.subplots(figsize=(16, 8))
bottom = np.zeros(len(top_15))

for col, name, color in zip(component_cols, component_names, colors):
    ax.bar(range(len(top_15)), top_15[col], bottom=bottom, label=name, color=color, alpha=0.8)
    bottom += top_15[col]

ax.set_xlabel('Zip Code')
ax.set_ylabel('Gap Score Points')
ax.set_title('Gap Score Components for Top 15 Highest-Need Zip Codes')
ax.set_xticks(range(len(top_15)))
ax.set_xticklabels([f"{row['zipCode']}\n({row['community']})" for _, row in top_15_with_names.iterrows()], 
                   rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

## Alternative Scoring Algorithms

Let's experiment with different weighting schemes:

In [ ]:
def calculate_gap_score_weighted(poverty_rate, population, service_count, nearest_distance, 
                                weights={'poverty': 0.4, 'population': 0.2, 'services': 0.25, 'distance': 0.15}):
    """
    Alternative algorithm with configurable weights
    """
    # Normalize each component to 0-1 scale
    
    # Poverty score (0-1, higher poverty = higher score)
    poverty_score = min(poverty_rate / 50, 1.0)  # Cap at 50% poverty
    
    # Population score (0-1, more people = higher score)
    population_score = min(population / 100000, 1.0)  # Cap at 100k population
    
    # Service score (0-1, fewer services = higher score)
    if service_count >= 10:
        service_score = 0
    else:
        service_score = (10 - service_count) / 10
    
    # Distance score (0-1, farther = higher score)
    if nearest_distance is None:
        distance_score = 1.0
    else:
        distance_score = min(nearest_distance / 10, 1.0)  # Cap at 10 miles
    
    # Weighted combination
    final_score = (
        poverty_score * weights['poverty'] +
        population_score * weights['population'] +
        service_score * weights['services'] +
        distance_score * weights['distance']
    )
    
    return final_score * 100  # Scale to 0-100

def calculate_gap_score_equity_focused(poverty_rate, population, service_count, nearest_distance):
    """
    Equity-focused algorithm that heavily weights poverty and access
    """
    return calculate_gap_score_weighted(
        poverty_rate, population, service_count, nearest_distance,
        weights={'poverty': 0.6, 'population': 0.1, 'services': 0.2, 'distance': 0.1}
    )

def calculate_gap_score_capacity_focused(poverty_rate, population, service_count, nearest_distance):
    """
    Capacity-focused algorithm that emphasizes population and services
    """
    return calculate_gap_score_weighted(
        poverty_rate, population, service_count, nearest_distance,
        weights={'poverty': 0.2, 'population': 0.4, 'services': 0.3, 'distance': 0.1}
    )

# Calculate alternative scores
valid_data['gapScore_weighted'] = valid_data.apply(
    lambda row: calculate_gap_score_weighted(
        row['povertyRate'], row['population'], row['serviceCount'], row['nearestDistance']
    ), axis=1
)

valid_data['gapScore_equity'] = valid_data.apply(
    lambda row: calculate_gap_score_equity_focused(
        row['povertyRate'], row['population'], row['serviceCount'], row['nearestDistance']
    ), axis=1
)

valid_data['gapScore_capacity'] = valid_data.apply(
    lambda row: calculate_gap_score_capacity_focused(
        row['povertyRate'], row['population'], row['serviceCount'], row['nearestDistance']
    ), axis=1
)

print("Alternative Scoring Comparison:")
for col in ['gapScore', 'gapScore_weighted', 'gapScore_equity', 'gapScore_capacity']:
    print(f"{col}: mean={valid_data[col].mean():.1f}, std={valid_data[col].std():.1f}")

# Correlation between different scoring methods
score_correlations = valid_data[[
    'gapScore', 'gapScore_weighted', 'gapScore_equity', 'gapScore_capacity'
]].corr()

print("\nCorrelations between scoring methods:")
print(score_correlations.round(3))

In [ ]:
# Compare scoring methods visually
fig, axes = plt.subplots(2, 2, figsize=(16, 12))
fig.suptitle('Comparison of Gap Scoring Algorithms', fontsize=16, fontweight='bold')

algorithms = [
    ('gapScore', 'Current Algorithm'),
    ('gapScore_weighted', 'Balanced Weighted'),
    ('gapScore_equity', 'Equity-Focused'),
    ('gapScore_capacity', 'Capacity-Focused')
]

for i, (col, title) in enumerate(algorithms):
    ax = axes[i//2, i%2]
    ax.hist(valid_data[col], bins=20, alpha=0.7, edgecolor='black')
    ax.axvline(valid_data[col].mean(), color='red', linestyle='--', 
               label=f'Mean: {valid_data[col].mean():.1f}')
    ax.set_xlabel('Gap Score')
    ax.set_ylabel('Number of Zip Codes')
    ax.set_title(title)
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Scatter plot matrix
fig, ax = plt.subplots(figsize=(12, 10))
scatter_data = valid_data[['gapScore', 'gapScore_weighted', 'gapScore_equity', 'gapScore_capacity']]
scatter_data.columns = ['Current', 'Weighted', 'Equity', 'Capacity']
pd.plotting.scatter_matrix(scatter_data, alpha=0.6, figsize=(12, 10), diagonal='hist', ax=ax)
plt.suptitle('Gap Score Method Correlations', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## Ranking Analysis

How do different algorithms rank zip codes differently?

In [ ]:
# Calculate rankings for each method
ranking_data = valid_data[['zipCode', 'community', 'povertyRate', 'population', 'serviceCount']].copy()

for col, name in algorithms:
    ranking_data[f'{name}_rank'] = valid_data[col].rank(ascending=False, method='min')
    ranking_data[f'{name}_score'] = valid_data[col]

# Top 20 comparison
print("Top 20 Zip Codes by Different Algorithms:")
print("\nCurrent Algorithm Top 20:")
current_top20 = ranking_data[ranking_data['Current Algorithm_rank'] <= 20].sort_values('Current Algorithm_rank')
print(current_top20[['zipCode', 'community', 'Current Algorithm_rank', 'Current Algorithm_score']].to_string(index=False))

print("\nEquity-Focused Algorithm Top 20:")
equity_top20 = ranking_data[ranking_data['Equity-Focused_rank'] <= 20].sort_values('Equity-Focused_rank')
print(equity_top20[['zipCode', 'community', 'Equity-Focused_rank', 'Equity-Focused_score']].to_string(index=False))

# Find zip codes that rank very differently
ranking_data['rank_difference'] = abs(
    ranking_data['Current Algorithm_rank'] - ranking_data['Equity-Focused_rank']
)

print("\nZip codes with biggest ranking differences (Current vs Equity-Focused):")
big_differences = ranking_data.nlargest(10, 'rank_difference')[[
    'zipCode', 'community', 'povertyRate', 'Current Algorithm_rank', 
    'Equity-Focused_rank', 'rank_difference'
]]
print(big_differences.to_string(index=False))

## Interactive Analysis Functions

Create functions to test custom weight combinations:

In [ ]:
def test_custom_weights(poverty_weight=0.4, population_weight=0.2, 
                       service_weight=0.25, distance_weight=0.15):
    """
    Test a custom weighting scheme
    """
    # Ensure weights sum to 1
    total_weight = poverty_weight + population_weight + service_weight + distance_weight
    if abs(total_weight - 1.0) > 0.01:
        print(f"Warning: Weights sum to {total_weight:.3f}, normalizing...")
        poverty_weight /= total_weight
        population_weight /= total_weight
        service_weight /= total_weight
        distance_weight /= total_weight
    
    weights = {
        'poverty': poverty_weight,
        'population': population_weight,
        'services': service_weight,
        'distance': distance_weight
    }
    
    # Calculate scores
    custom_scores = valid_data.apply(
        lambda row: calculate_gap_score_weighted(
            row['povertyRate'], row['population'], row['serviceCount'], 
            row['nearestDistance'], weights
        ), axis=1
    )
    
    # Create results dataframe
    results = valid_data[['zipCode', 'community', 'povertyRate', 'population', 
                         'serviceCount', 'nearestDistance']].copy()
    results['customScore'] = custom_scores
    results['rank'] = custom_scores.rank(ascending=False, method='min')
    
    print(f"Custom Weights: Poverty={poverty_weight:.2f}, Population={population_weight:.2f}, "
          f"Services={service_weight:.2f}, Distance={distance_weight:.2f}")
    print(f"Score Statistics: Mean={custom_scores.mean():.1f}, Std={custom_scores.std():.1f}")
    
    print("\nTop 15 Zip Codes:")
    top_15 = results.nsmallest(15, 'rank')
    print(top_15[['zipCode', 'community', 'customScore', 'rank']].to_string(index=False))
    
    return results

# Test some examples
print("=" * 60)
print("EXAMPLE 1: Poverty-Heavy Weighting")
print("=" * 60)
poverty_heavy = test_custom_weights(poverty_weight=0.7, population_weight=0.1, 
                                   service_weight=0.15, distance_weight=0.05)

print("\n" + "=" * 60)
print("EXAMPLE 2: Service-Access Focused")
print("=" * 60)
access_focused = test_custom_weights(poverty_weight=0.3, population_weight=0.1, 
                                    service_weight=0.4, distance_weight=0.2)

## Sensitivity Analysis

How sensitive are the results to parameter changes?

In [ ]:
def sensitivity_analysis():
    """
    Test how small changes in weights affect rankings
    """
    base_weights = {'poverty': 0.4, 'population': 0.2, 'services': 0.25, 'distance': 0.15}
    
    # Calculate base scores and rankings
    base_scores = valid_data.apply(
        lambda row: calculate_gap_score_weighted(
            row['povertyRate'], row['population'], row['serviceCount'], 
            row['nearestDistance'], base_weights
        ), axis=1
    )
    base_ranks = base_scores.rank(ascending=False, method='min')
    
    sensitivity_results = []
    
    # Test variations in each weight
    for weight_name in base_weights.keys():
        for delta in [-0.1, -0.05, 0.05, 0.1]:
            test_weights = base_weights.copy()
            test_weights[weight_name] += delta
            
            # Normalize other weights
            remaining_weight = 1 - test_weights[weight_name]
            other_weights = {k: v for k, v in base_weights.items() if k != weight_name}
            other_total = sum(other_weights.values())
            
            for k in other_weights:
                test_weights[k] = (other_weights[k] / other_total) * remaining_weight
            
            # Calculate new scores
            test_scores = valid_data.apply(
                lambda row: calculate_gap_score_weighted(
                    row['povertyRate'], row['population'], row['serviceCount'], 
                    row['nearestDistance'], test_weights
                ), axis=1
            )
            test_ranks = test_scores.rank(ascending=False, method='min')
            
            # Calculate rank correlation
            rank_correlation = base_ranks.corr(test_ranks)
            
            # Count how many of top 20 changed
            base_top20 = set(base_ranks[base_ranks <= 20].index)
            test_top20 = set(test_ranks[test_ranks <= 20].index)
            top20_overlap = len(base_top20.intersection(test_top20))
            
            sensitivity_results.append({
                'weight_changed': weight_name,
                'delta': delta,
                'new_weight': test_weights[weight_name],
                'rank_correlation': rank_correlation,
                'top20_overlap': top20_overlap
            })
    
    sensitivity_df = pd.DataFrame(sensitivity_results)
    
    print("Sensitivity Analysis Results:")
    print("(How much do rankings change with small weight adjustments?)")
    print()
    
    for weight in base_weights.keys():
        weight_data = sensitivity_df[sensitivity_df['weight_changed'] == weight]
        print(f"\n{weight.title()} Weight Sensitivity:")
        for _, row in weight_data.iterrows():
            print(f"  {row['delta']:+.2f} -> {row['new_weight']:.2f}: "
                  f"Rank correlation = {row['rank_correlation']:.3f}, "
                  f"Top-20 overlap = {row['top20_overlap']}/20")
    
    return sensitivity_df

sensitivity_results = sensitivity_analysis()

## Summary and Recommendations

In [ ]:
print("=" * 80)
print("SAN DIEGO HOMELESSNESS SERVICE GAP ANALYSIS - SUMMARY")
print("=" * 80)

print(f"\n📊 DATA OVERVIEW:")
print(f"   • {len(valid_data)} zip codes analyzed")
print(f"   • {len(services_clean)} homeless service locations")
print(f"   • Poverty rates range: {valid_data['povertyRate'].min():.1f}% - {valid_data['povertyRate'].max():.1f}%")
print(f"   • Population range: {valid_data['population'].min():,.0f} - {valid_data['population'].max():,.0f}")

print(f"\n🎯 CURRENT ALGORITHM PERFORMANCE:")
print(f"   • Average gap score: {valid_data['gapScore'].mean():.1f}/100")
print(f"   • High-need areas (>80): {(valid_data['gapScore'] > 80).sum()} zip codes")
print(f"   • Top gap score: {valid_data['gapScore'].max():.1f} (Zip {valid_data.loc[valid_data['gapScore'].idxmax(), 'zipCode']})")

print(f"\n🏆 HIGHEST NEED ZIP CODES (Current Algorithm):")
top_5_current = valid_data.nlargest(5, 'gapScore')
for _, row in top_5_current.iterrows():
    print(f"   • {row['zipCode']} ({row['community']}): {row['gapScore']:.1f} points")
    print(f"     - Poverty: {row['povertyRate']:.1f}%, Population: {row['population']:,.0f}, Services: {row['serviceCount']}")

print(f"\n⚖️ COMPONENT ANALYSIS:")
components = calculate_score_components(valid_data)
total_avg = components['total_score'].mean()
print(f"   • Poverty component: {components['poverty_component'].mean():.1f} points ({components['poverty_component'].mean()/total_avg*100:.1f}% of total)")
print(f"   • Population component: {components['population_component'].mean():.1f} points ({components['population_component'].mean()/total_avg*100:.1f}% of total)")
print(f"   • Service component: {components['service_component'].mean():.1f} points ({components['service_component'].mean()/total_avg*100:.1f}% of total)")
print(f"   • Distance component: {components['distance_component'].mean():.1f} points ({components['distance_component'].mean()/total_avg*100:.1f}% of total)")

print(f"\n🔄 ALTERNATIVE ALGORITHMS:")
print(f"   • Equity-focused (60% poverty weight):")
top_equity = valid_data.nlargest(3, 'gapScore_equity')
for _, row in top_equity.iterrows():
    print(f"     - {row['zipCode']} ({row['community']}): {row['gapScore_equity']:.1f} points")

print(f"\n   • Capacity-focused (40% population weight):")
top_capacity = valid_data.nlargest(3, 'gapScore_capacity')
for _, row in top_capacity.iterrows():
    print(f"     - {row['zipCode']} ({row['community']}): {row['gapScore_capacity']:.1f} points")

print(f"\n💡 KEY INSIGHTS:")
print(f"   • {(valid_data['serviceCount'] == 0).sum()} zip codes have NO homeless services")
print(f"   • Average distance to nearest service: {valid_data['nearestDistance'].mean():.1f} miles")
print(f"   • Poverty and gap scores are strongly correlated (r = {valid_data['povertyRate'].corr(valid_data['gapScore']):.2f})")
print(f"   • Population size has moderate impact (r = {valid_data['population'].corr(valid_data['gapScore']):.2f})")

print(f"\n🎛️ TUNING RECOMMENDATIONS:")
print(f"   • Current algorithm balances multiple factors well")
print(f"   • For equity focus: Increase poverty weight to 0.6+")
print(f"   • For capacity planning: Increase population weight to 0.4+")
print(f"   • Algorithm is moderately sensitive to weight changes")
print(f"   • Consider adding demographic factors (veterans, seniors, youth)")

print("\n" + "=" * 80)

## Export Results

Save analysis results for further use:

In [ ]:
# Create comprehensive results dataset
export_data = valid_data[[
    'zipCode', 'community', 'population', 'povertyRate', 
    'serviceCount', 'nearestDistance', 'medianIncome'
]].copy()

export_data['gapScore_current'] = valid_data['gapScore']
export_data['gapScore_equity'] = valid_data['gapScore_equity']
export_data['gapScore_capacity'] = valid_data['gapScore_capacity']

# Add component breakdowns
components = calculate_score_components(valid_data)
export_data['poverty_component'] = components['poverty_component']
export_data['population_component'] = components['population_component']
export_data['service_component'] = components['service_component']
export_data['distance_component'] = components['distance_component']

# Add rankings
export_data['rank_current'] = valid_data['gapScore'].rank(ascending=False, method='min')
export_data['rank_equity'] = valid_data['gapScore_equity'].rank(ascending=False, method='min')
export_data['rank_capacity'] = valid_data['gapScore_capacity'].rank(ascending=False, method='min')

# Save to CSV
export_data.to_csv('service_gap_analysis_results.csv', index=False)
print("Results exported to 'service_gap_analysis_results.csv'")

# Display sample of export data
print("\nSample of exported data:")
print(export_data.head(10).to_string(index=False))

print(f"\n✅ Analysis complete! Check the exported CSV file for detailed results.")